# Algoritmos de optimización - Reto 3

Nombre: Ramón Murias Palomer

Github: <https://github.com/ramonmurias/Trabajos_Algoritmos_Optimizacion>

Implementar uno de los siguientes retos:
1. Mejorar la implementación de búsqueda local implementada en  clase sobre el TSP con otros operadores de vecindad.
2. Implementar el algoritmo de búsqueda tabú para el TSP de la AG3.
3. Mejorar la implementación de recocido simulado implementado en clase sobre el TSP eligiendo una generación de solución vecina con un grado de aleatoriedad menor (función genera_vecino_aleatorio()).
4. Mejorar la implementación de colonia de hormigas implementada en clase sobre el TSP mediante una elección de nodo que tenga en consideración una función de probabilidad que depende de las feromonas.
5. Mejorar la implementación del algoritmo genético propuesto para la resolución del TSP.

La fecha límite de entrega será el próximo día 19/07/2024 a las 23:59 horas.
El formato de entrega será el de Jupyter Notebook exportado a HTML. Se debe mostrar el resultado de ejecución de las celdas. Se debe mostrar el resultado de, al menos, dos pruebas de cada algoritmo implementado para condiciones de entrada diferentes. El código debe estar debidamene comentado. Se deberá adjuntar también el archivo con extensión IPYNB o un enlace a Github por si es necesaria la ejecución de código. Los distintos ficheros se comprimirán en formato *.ZIP o *.RAR para su entrega en el aula virtual.

In [58]:
!pip install requests    #Hacer llamadas http a paginas de la red
!pip install tsplib95    #Modulo para las instancias del problema del TSP

In [59]:
import urllib.request #Hacer llamadas http a paginas de la red
import tsplib95       #Modulo para las instancias del problema del TSP
import math           #Modulo de funciones matematicas. Se usa para exp
import random         #Para generar valores aleatorios
import copy           #Para hacer copias de estructuras de datos

#http://elib.zib.de/pub/mp-testdata/tsp/tsplib/
#Documentacion :
  # http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp95.pdf
  # https://tsplib95.readthedocs.io/en/stable/pages/usage.html
  # https://tsplib95.readthedocs.io/en/v0.6.1/modules.html
  # https://pypi.org/project/tsplib95/

#Descargamos el fichero de datos(Matriz de distancias)
file = "swiss42.tsp" ;
urllib.request.urlretrieve("http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/swiss42.tsp.gz", file + '.gz')
!gzip -d swiss42.tsp.gz     #Descomprimir el fichero de datos

#Coordendas 51-city problem (Christofides/Eilon)
#file = "eil51.tsp" ; urllib.request.urlretrieve("http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/eil51.tsp.gz", file)

#Coordenadas - 48 capitals of the US (Padberg/Rinaldi)
#file = "att48.tsp" ; urllib.request.urlretrieve("http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/att48.tsp.gz", file)


gzip: swiss42.tsp already exists; do you wish to overwrite (y or n)? ^C


In [60]:
#Carga de datos y generación de objeto problem
###############################################################################
problem = tsplib95.load(file)

#Nodos
Nodos = list(problem.get_nodes())

#Aristas
Aristas = list(problem.get_edges())

Aristas

[(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (0, 5),
 (0, 6),
 (0, 7),
 (0, 8),
 (0, 9),
 (0, 10),
 (0, 11),
 (0, 12),
 (0, 13),
 (0, 14),
 (0, 15),
 (0, 16),
 (0, 17),
 (0, 18),
 (0, 19),
 (0, 20),
 (0, 21),
 (0, 22),
 (0, 23),
 (0, 24),
 (0, 25),
 (0, 26),
 (0, 27),
 (0, 28),
 (0, 29),
 (0, 30),
 (0, 31),
 (0, 32),
 (0, 33),
 (0, 34),
 (0, 35),
 (0, 36),
 (0, 37),
 (0, 38),
 (0, 39),
 (0, 40),
 (0, 41),
 (1, 0),
 (1, 1),
 (1, 2),
 (1, 3),
 (1, 4),
 (1, 5),
 (1, 6),
 (1, 7),
 (1, 8),
 (1, 9),
 (1, 10),
 (1, 11),
 (1, 12),
 (1, 13),
 (1, 14),
 (1, 15),
 (1, 16),
 (1, 17),
 (1, 18),
 (1, 19),
 (1, 20),
 (1, 21),
 (1, 22),
 (1, 23),
 (1, 24),
 (1, 25),
 (1, 26),
 (1, 27),
 (1, 28),
 (1, 29),
 (1, 30),
 (1, 31),
 (1, 32),
 (1, 33),
 (1, 34),
 (1, 35),
 (1, 36),
 (1, 37),
 (1, 38),
 (1, 39),
 (1, 40),
 (1, 41),
 (2, 0),
 (2, 1),
 (2, 2),
 (2, 3),
 (2, 4),
 (2, 5),
 (2, 6),
 (2, 7),
 (2, 8),
 (2, 9),
 (2, 10),
 (2, 11),
 (2, 12),
 (2, 13),
 (2, 14),
 (2, 15),
 (2, 16),
 (2, 17),
 (2, 18),


In [61]:

#Funcionas basicas
###############################################################################

#Se genera una solucion aleatoria con comienzo en en el nodo 0
def crear_solucion(Nodos):
  solucion = [Nodos[0]]
  for n in Nodos[1:]:
    solucion = solucion + [random.choice(list(set(Nodos) - set({Nodos[0]}) - set(solucion)))]
  return solucion

#Devuelve la distancia entre dos nodos
def distancia(a,b, problem):
  return problem.get_weight(a,b)

#Devuelve la distancia total de una trayectoria/solucion
def distancia_total(solucion, problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problem)
  return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)

sol_temporal = crear_solucion(Nodos)

distancia_total(sol_temporal, problem), sol_temporal

(4944,
 [0,
  4,
  37,
  30,
  29,
  24,
  17,
  40,
  2,
  7,
  27,
  28,
  13,
  12,
  41,
  19,
  31,
  14,
  26,
  23,
  33,
  11,
  18,
  20,
  15,
  35,
  1,
  9,
  3,
  36,
  32,
  25,
  10,
  38,
  39,
  16,
  6,
  22,
  8,
  34,
  5,
  21])

1. Mejorar la implementación de búsqueda local implementada en  clase sobre el TSP con otros operadores de vecindad.

In [62]:

###############################################################################
# BUSQUEDA LOCAL
###############################################################################
def genera_vecina(solucion):
  #Generador de soluciones vecinas: 2-opt (intercambiar 2 nodos) Si hay N nodos se generan (N-1)x(N-2)/2 soluciones
  #Se puede modificar para aplicar otros generadores distintos que 2-opt
  #print(solucion)
  mejor_solucion = []
  mejor_distancia = 10e100
  for i in range(1,len(solucion)-1):          #Recorremos todos los nodos en bucle doble para evaluar todos los intercambios 2-opt
    for j in range(i+1, len(solucion)):

      #Se genera una nueva solución intercambiando los dos nodos i,j:
      #  (usamos el operador + que para listas en python las concatena) : ej.: [1,2] + [3] = [1,2,3]
      vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]

      #Se evalua la nueva solución ...
      distancia_vecina = distancia_total(vecina, problem)

      #... para guardarla si mejora las anteriores
      if distancia_vecina <= mejor_distancia:
        mejor_distancia = distancia_vecina
        mejor_solucion = vecina
  return mejor_solucion

#Busqueda Local:
#  - Sobre el operador de vecindad 2-opt(funcion genera_vecina)
#  - Sin criterio de parada, se para cuando no es posible mejorar.
def busqueda_local(problem):
  mejor_solucion = []

  #Generar una solucion inicial de referencia(aleatoria)
  solucion_referencia = crear_solucion(Nodos)
  mejor_distancia = distancia_total(solucion_referencia, problem)

  iteracion=0             #Un contador para saber las iteraciones que hacemos
  while(1):
    iteracion +=1         #Incrementamos el contador
    #print('#',iteracion)

    #Obtenemos la mejor vecina ...
    vecina = genera_vecina(solucion_referencia)

    #... y la evaluamos para ver si mejoramos respecto a lo encontrado hasta el momento
    distancia_vecina = distancia_total(vecina, problem)

    #Si no mejoramos hay que terminar. Hemos llegado a un minimo local(según nuestro operador de vencindad 2-opt)
    if distancia_vecina < mejor_distancia:
      #mejor_solucion = copy.deepcopy(vecina)   #Con copia profunda. Las copias en python son por referencia
      mejor_solucion = vecina                   #Guarda la mejor solución encontrada
      mejor_distancia = distancia_vecina

    else:
      print("En la iteracion ", iteracion, ", la mejor solución encontrada es:" , mejor_solucion)
      print("Distancia     :" , mejor_distancia)
      return mejor_solucion

    solucion_referencia = vecina


sol = busqueda_local(problem )

In [65]:
# Intercambia dos nodos en la solución
def swap(solucion, i, j):
    vecina = solucion[:]
    vecina[i], vecina[j] = vecina[j], vecina[i]
    return vecina

# Mueve un nodo a una nueva posición en la solución
def insert(solucion, i, j):
    vecina = solucion[:]
    nodo = vecina.pop(i)
    vecina.insert(j, nodo)
    return vecina

# Invertimos una subruta en la solución
def two_opt(solucion, i, j):
    vecina = solucion[:i] + solucion[i:j+1][::-1] + solucion[j+1:]
    return vecina

# Generador de soluciones vecinas utilizando diferentes operadores
def genera_vecina_reto03(solucion, operador):
    mejor_solucion = []
    mejor_distancia = 10e100

    # Recorremos todos los nodos en bucle doble para evaluar todos los intercambios según el operador
    for i in range(1, len(solucion) - 1):
        for j in range(i + 1, len(solucion)):
            if operador == 'swap':
                vecina = swap(solucion, i, j)
            elif operador == 'insert':
                vecina = insert(solucion, i, j)
            elif operador == '2-opt':
                vecina = two_opt(solucion, i, j)
            else:
                raise ValueError("Operador no válido")

            # Evaluar la nueva solución
            distancia_vecina = distancia_total(vecina, problem)

            # Guardar la mejor solución encontrada
            if distancia_vecina < mejor_distancia:
                mejor_distancia = distancia_vecina
                mejor_solucion = vecina
    return mejor_solucion

# Búsqueda Local utilizando diferentes operadores de vecindad
def busqueda_local_reto03(problem, operador='2-opt'):
    mejor_solucion = []

    # Generar una solución inicial de referencia (aleatoria)
    solucion_referencia = crear_solucion(Nodos)
    mejor_distancia = distancia_total(solucion_referencia, problem)

    iteracion = 0  # Contador de iteraciones
    while True:
        iteracion += 1  # Incrementar el contador de iteraciones

        # Obtener la mejor vecina evaluando todas las posibles intercambios de nodos
        vecina = genera_vecina_reto03(solucion_referencia, operador)

        # Evaluar la vecina generada
        distancia_vecina = distancia_total(vecina, problem)

        # Si la vecina mejora la mejor solución encontrada hasta ahora, se actualiza
        if distancia_vecina < mejor_distancia:
            mejor_solucion = vecina  # Guardar la mejor solución encontrada
            mejor_distancia = distancia_vecina
        else:
            # Si no mejora, se ha llegado a un mínimo local y el algoritmo termina
            print(f"En la iteración {iteracion}, la mejor solución encontrada es: {mejor_solucion}")
            print(f"Distancia: {mejor_distancia}")
            return mejor_solucion

        # Actualizar la solución de referencia para la próxima iteración
        solucion_referencia = vecina

sol = busqueda_local_reto03(problem, 'swap')
sol = busqueda_local_reto03(problem, 'insert')
sol = busqueda_local_reto03(problem, '2-opt')

En la iteración 43, la mejor solución encontrada es: [0, 27, 2, 3, 6, 5, 15, 37, 17, 31, 20, 33, 34, 30, 38, 22, 39, 21, 24, 40, 13, 19, 16, 14, 28, 8, 41, 23, 9, 29, 32, 35, 36, 7, 4, 10, 25, 11, 12, 18, 26, 1]
Distancia: 1884
En la iteración 31, la mejor solución encontrada es: [0, 1, 6, 9, 40, 24, 22, 38, 28, 27, 2, 12, 11, 25, 8, 30, 32, 34, 33, 20, 35, 36, 31, 17, 7, 37, 15, 16, 14, 19, 13, 5, 26, 18, 10, 41, 23, 21, 39, 29, 4, 3]
Distancia: 1680
En la iteración 36, la mejor solución encontrada es: [0, 32, 34, 33, 20, 35, 36, 31, 17, 7, 37, 15, 16, 14, 19, 13, 5, 26, 18, 12, 11, 25, 10, 29, 8, 41, 23, 9, 21, 40, 24, 39, 22, 38, 30, 28, 27, 2, 3, 4, 6, 1]
Distancia: 1324


2. Implementar el algoritmo de búsqueda tabú para el TSP de la AG3.

In [66]:
def generar_vecinos_tabu(solucion):
    vecinos = []
    for i in range(1, len(solucion) - 1):
        for j in range(i + 1, len(solucion)):
            vecino = swap(solucion, i, j)
            vecinos.append(vecino)
    return vecinos

def busqueda_tabu_reto03(problem, tenencia_tabu, max_iter):
    # Generar una solución inicial de referencia (aleatoria)
    solucion_actual = crear_solucion(Nodos)
    mejor_solucion = copy.deepcopy(solucion_actual)
    mejor_distancia = distancia_total(solucion_actual, problem)

    # Lista tabú
    lista_tabu = []

    for iteracion in range(max_iter):

        # Generar todos los vecinos de la solución actual
        vecinos = generar_vecinos_tabu(solucion_actual)

        # Evaluar y seleccionar el mejor vecino no tabú
        mejor_vecino = None
        mejor_vecino_distancia = float('inf')
        for vecino in vecinos:
            if vecino not in lista_tabu:
                dist = distancia_total(vecino, problem)
                if dist < mejor_vecino_distancia:
                    mejor_vecino = vecino
                    mejor_vecino_distancia = dist

        # Actualizar la lista tabú
        if len(lista_tabu) >= tenencia_tabu:
            lista_tabu.pop(0)
        lista_tabu.append(mejor_vecino)

        # Actualizar la solución actual
        solucion_actual = mejor_vecino

        # Actualizar la mejor solución encontrada
        if mejor_vecino_distancia < mejor_distancia:
            mejor_solucion = mejor_vecino
            mejor_distancia = mejor_vecino_distancia


    return mejor_solucion, mejor_distancia

# Ejecutar la búsqueda tabú
tenencia_tabu = 7  # Longitud de la lista tabú
max_iter = 100  # Número máximo de iteraciones
solucion_final, distancia_final = busqueda_tabu_reto03(problem, tenencia_tabu, max_iter)

print("Solución final:", solucion_final)
print("Distancia final:", distancia_final)

Solución final: [0, 3, 27, 2, 28, 30, 29, 8, 10, 11, 25, 41, 23, 21, 39, 9, 18, 7, 15, 16, 14, 19, 13, 12, 40, 24, 22, 38, 35, 36, 37, 17, 31, 20, 33, 34, 32, 4, 26, 5, 6, 1]
Distancia final: 1808


3. Mejorar la implementación de recocido simulado implementado en clase sobre el TSP eligiendo una generación de solución vecina con un grado de aleatoriedad menor (función genera_vecino_aleatorio()).

In [67]:
###############################################################################
# SIMULATED ANNEALING
###############################################################################

#Funcion de probabilidad para aceptar peores soluciones
def probabilidad(T,d):
  if random.random() <  math.exp( -1*d / T)  :
    return True
  else:
    return False

#Funcion de descenso de temperatura
def bajar_temperatura(T):
  return T*0.99

# Generador de 1 solución vecina 2-opt con menor aleatoriedad
def genera_vecina_menos_aleatorio(solucion):
    # Escoger un par de índices aleatorios pero no completamente aleatorios
    i = random.randint(1, len(solucion) - 2)
    j = random.randint(i + 1, len(solucion) - 1)

    # Intercambiar los nodos seleccionados
    vecina = solucion[:]
    vecina[i], vecina[j] = vecina[j], vecina[i]
    return vecina

def recocido_simulado(problem, TEMPERATURA ):
  #problem = datos del problema
  #T = Temperatura

  solucion_referencia = crear_solucion(Nodos)
  distancia_referencia = distancia_total(solucion_referencia, problem)

  mejor_solucion = []             #x* del seudocodigo
  mejor_distancia = 10e100        #F* del seudocodigo


  N=0
  while TEMPERATURA > .0001:
    N+=1
    #Genera una solución vecina
    vecina = genera_vecina_menos_aleatorio(solucion_referencia)

    #Calcula su valor(distancia)
    distancia_vecina = distancia_total(vecina, problem)

    #Si es la mejor solución de todas se guarda(siempre!!!)
    if distancia_vecina < mejor_distancia:
        mejor_solucion = vecina
        mejor_distancia = distancia_vecina

    #Si la nueva vecina es mejor se cambia
    #Si es peor se cambia según una probabilidad que depende de T y delta(distancia_referencia - distancia_vecina)
    if distancia_vecina < distancia_referencia or probabilidad(TEMPERATURA, abs(distancia_referencia - distancia_vecina) ) :
      #solucion_referencia = copy.deepcopy(vecina)
      solucion_referencia = vecina
      distancia_referencia = distancia_vecina

    #Bajamos la temperatura
    TEMPERATURA = bajar_temperatura(TEMPERATURA)

  print("La mejor solución encontrada es " , end="")
  print(mejor_solucion)
  print("con una distancia total de " , end="")
  print(mejor_distancia)
  return mejor_solucion

sol  = recocido_simulado(problem, 10000000)

La mejor solución encontrada es [0, 32, 34, 29, 10, 12, 11, 8, 9, 2, 3, 6, 1, 37, 15, 5, 4, 14, 16, 19, 13, 26, 27, 28, 30, 38, 22, 39, 21, 24, 40, 23, 41, 25, 18, 7, 31, 20, 33, 35, 36, 17]
con una distancia total de 1943


4. Mejorar la implementación de colonia de hormigas implementada en clase sobre el TSP mediante una elección de nodo que tenga en consideración una función de probabilidad que depende de las feromonas.

In [68]:
def distancia(a, b, problem):
    """Calcula la distancia entre dos nodos."""
    return problem.get_weight(a, b)  # Use get_weight to get distance from distance matrix

# Función auxiliar para calcular la distancia total de una solución (ruta)
def distancia_total(solucion, problem):
    """Calcula la distancia total de una solución (ruta)."""
    dist_total = 0
    for i in range(len(solucion)):
        dist_total += distancia(solucion[i], solucion[(i + 1) % len(solucion)], problem)
    return dist_total

# Función para añadir un nodo basado en la probabilidad de feromonas y distancia
def add_nodo(problem, hormiga, feromonas, alfa=1.0, beta=1.0):
    """Selecciona un nuevo nodo para añadir a la hormiga basado en feromonas y distancia."""
    nodos = list(problem.get_nodes())
    no_visitados = list(set(nodos) - set(hormiga))

    # Verificar que haya nodos no visitados
    if not no_visitados:
        return hormiga[-1]  # Volver al último nodo si no hay nodos no visitados

    # Calcular las probabilidades de selección basadas en feromonas y distancia
    sumatoria_probabilidades = 0
    probabilidades = []

    for nodo in no_visitados:
        probabilidad = (feromonas[hormiga[-1]][nodo] ** alfa) * ((1.0 / distancia(hormiga[-1], nodo, problem)) ** beta)
        probabilidades.append((nodo, probabilidad))
        sumatoria_probabilidades += probabilidad

    # Normalizar probabilidades y seleccionar el nodo
    probabilidades = [(nodo, probabilidad / sumatoria_probabilidades) for nodo, probabilidad in probabilidades]
    seleccionado = random.choices(range(len(probabilidades)), weights=[prob for nodo, prob in probabilidades])[0]

    return probabilidades[seleccionado][0]

# Función para incrementar las feromonas en la solución encontrada por la hormiga
def incrementa_feromona(problem, feromonas, hormiga, Q):
    """Incrementa las feromonas en el camino de la hormiga según la calidad de la solución."""
    for i in range(len(hormiga) - 1):
        feromonas[hormiga[i]][hormiga[i + 1]] += Q / distancia_total(hormiga, problem)
    return feromonas

# Función para evaporar las feromonas en todas las aristas
def evaporar_feromonas(feromonas, rho):
    """Evapora las feromonas en todas las aristas, manteniendo un mínimo de 1."""
    for i in range(len(feromonas)):
        for j in range(len(feromonas)):
            feromonas[i][j] *= rho
            if feromonas[i][j] < 1.0:
                feromonas[i][j] = 1.0
    return feromonas

# Función principal para el algoritmo de colonia de hormigas
def colonia_hormigas(problem, N, max_iter=1000, alfa=1.0, beta=1.0, rho=0.3, Q=100):
    """Implementación del algoritmo de colonia de hormigas para resolver el TSP."""

    # Número de nodos y aristas
    num_nodos = len(list(problem.get_nodes()))
    nodos = list(problem.get_nodes()) # Get list of nodes

    # Matriz de feromonas inicial (1 en todas las aristas)
    feromonas = [[1.0] * num_nodos for _ in range(num_nodos)]

    # Lista para almacenar las soluciones encontradas por cada hormiga
    hormigas = [[] for _ in range(N)]

    # Iteraciones del algoritmo
    for iteracion in range(max_iter):
        # Construir soluciones para cada hormiga
        for h in range(N):
            # Initialize the ant's path with a random starting node
            hormigas[h].append(random.choice(nodos)) # Start each ant at a random node

            # Construir camino para la hormiga h
            for i in range(num_nodos - 1):
                nuevo_nodo = add_nodo(problem, hormigas[h], feromonas, alfa, beta)
                hormigas[h].append(nuevo_nodo)

            # Incrementar feromonas en el camino de la hormiga h
            feromonas = incrementa_feromona(problem, feromonas, hormigas[h], Q)

            # Evaporar feromonas en todas las aristas
            feromonas = evaporar_feromonas(feromonas, rho)

        # Seleccionar la mejor solución encontrada hasta el momento
        mejor_solucion = []
        mejor_distancia = float('inf')

        for h in range(N):
            distancia_actual = distancia_total(hormigas[h], problem)
            if distancia_actual < mejor_distancia:
                mejor_solucion = hormigas[h]
                mejor_distancia = distancia_actual

    return mejor_solucion, mejor_distancia
# Ejecutar el algoritmo de colonia de hormigas
solucion_final, distancia_final = colonia_hormigas(problem, N=50, max_iter=10)

print("Mejor solución encontrada:", solucion_final)
print("Distancia de la mejor solución:", distancia_final)

Mejor solución encontrada: [22, 34, 28, 3, 4, 30, 14, 16, 15, 37, 7, 1, 5, 6, 41, 40, 24, 27, 17, 31, 0, 9, 23, 11, 39, 21, 35, 36, 19, 18, 12, 8, 10, 32, 2, 33, 38, 20, 25, 29, 26, 13, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 

5. Mejorar la implementación del algoritmo genético propuesto para la resolución del TSP.

In [69]:
# Genera una poblacion inicial de soluciones de tamaño N
def generar_poblacion(Nodos, N):
    """Genera una población inicial de soluciones."""
    return [crear_solucion(Nodos) for _ in range(N)]

# Evalua la población y devuelve el mejor individuo
def Evaluar_Poblacion(poblacion, problem):
    """Evalúa la población y devuelve el mejor individuo."""
    mejor_solucion = None
    mejor_distancia = float('inf')
    for p in poblacion:
        distancia_referencia = distancia_total(p, problem)
        if distancia_referencia < mejor_distancia:
            mejor_solucion = p
            mejor_distancia = distancia_referencia
    return mejor_solucion, mejor_distancia

# Función de cruce. Recibe una poblacion (lista de soluciones) y devuelve la población ampliada con los hijos.
def Cruzar(poblacion, mutacion, problem):
    """Función de cruce que produce hijos a partir de la población actual."""
    poblacion_copia = copy.deepcopy(poblacion)
    poblacion_final = copy.deepcopy(poblacion)

    while len(poblacion_copia) > 1:
        padre1, padre2 = random.sample(poblacion_copia, 2)
        poblacion_copia.remove(padre1)
        poblacion_copia.remove(padre2)
        poblacion_final.extend(Descendencia([padre1, padre2], problem, mutacion))
    return poblacion_final

# Función para generar hijos a partir de 2 padres
def Descendencia(padres, problem, mutacion):
    """Genera hijos a partir de dos padres usando cruce de 1-punto y aplica mutación."""
    pc = random.sample(range(1, len(padres[0])), 1)[0]
    hijo1 = Factibilizar(padres[0][:pc] + padres[1][pc:], problem)
    hijo2 = Factibilizar(padres[1][:pc] + padres[0][pc:], problem)
    return [hijo1, hijo2, Mutar(hijo1, mutacion), Mutar(hijo2, mutacion)]

# Función para hacer factible una solución (evitar nodos repetidos)
def Factibilizar(solucion, problem):
    """Asegura que la solución sea factible (sin nodos repetidos)."""
    Nodos = list(problem.get_nodes())
    nodos_desaparecidos = list(set(Nodos) - set(solucion))
    for i in range(len(solucion)):
        if solucion[i] in solucion[:i]:
            solucion[i] = nodos_desaparecidos.pop(0)
    return solucion

# Función de mutación. Intercambia dos nodos con cierta probabilidad
def Mutar(solucion, mutacion):
    """Aplica mutación intercambiando dos nodos con cierta probabilidad."""
    if random.random() < mutacion:
        sel1, sel2 = sorted(random.sample(set(solucion) - {solucion[0]}, 2))
        return solucion[:sel1] + [solucion[sel2]] + solucion[sel1 + 1:sel2] + [solucion[sel1]] + solucion[sel2 + 1:]
    else:
        return solucion[::]

# Función de selección de la población. Aplica elitismo y selección aleatoria para mantener la población estable
def Seleccionar(problem, poblacion, N, elitismo):
    """Selecciona la población basada en elitismo y selección aleatoria."""
    poblacion_ordenada = sorted([[distancia_total(solucion, problem), solucion] for solucion in poblacion], key=lambda x: x[0])
    return [x[1] for x in poblacion_ordenada][:int(N * elitismo)] + random.sample([x[1] for x in poblacion_ordenada][int(N * elitismo):], int(N * (1 - elitismo)))

# Función principal del algoritmo genético
def algoritmo_genetico(problem=problem, N=100, mutacion=0.15, elitismo=0.1, generaciones=100):
    """Implementación del algoritmo genético para resolver el TSP."""

    # Genera la población inicial
    Nodos = list(problem.get_nodes())
    poblacion = generar_poblacion(Nodos, N)

    # Evaluar la población inicial
    (mejor_solucion, mejor_distancia) = Evaluar_Poblacion(poblacion, problem)

    # Condición de parada
    parar = False
    n = 0

    # Iniciar el ciclo de generaciones
    while not parar:
        # Cruzar la población (incluye mutación)
        poblacion = Cruzar(poblacion, mutacion, problem)

        # Seleccionar la población
        poblacion = Seleccionar(problem, poblacion, N, elitismo)

        # Evaluar la nueva población
        (mejor_solucion, mejor_distancia) = Evaluar_Poblacion(poblacion, problem)

        print("Generación #", n, "\nLa mejor solución es:", mejor_solucion, "\ncon distancia:", mejor_distancia, "\n")

        # Criterio de parada por número de generaciones
        if n == generaciones:
            parar = True
        n += 1

    return mejor_solucion

# Ejecutar el algoritmo genético
sol = algoritmo_genetico(problem=problem, N=500, mutacion=0.3, elitismo=0.4, generaciones=250)

<ipython-input-69-1414776ad201>:53: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  sel1, sel2 = sorted(random.sample(set(solucion) - {solucion[0]}, 2))


Generación # 0 
La mejor solución es: [0, 35, 22, 30, 15, 10, 29, 27, 3, 33, 36, 6, 4, 41, 39, 21, 16, 7, 32, 17, 31, 13, 11, 8, 34, 18, 12, 28, 2, 37, 1, 5, 38, 14, 20, 9, 40, 24, 23, 25, 19, 26] 
con distancia: 3912 

Generación # 1 
La mejor solución es: [0, 17, 36, 37, 14, 6, 5, 33, 30, 19, 11, 12, 2, 34, 21, 40, 27, 1, 29, 41, 15, 16, 32, 31, 35, 20, 28, 38, 22, 9, 10, 26, 3, 8, 13, 23, 18, 24, 39, 4, 25, 7] 
con distancia: 3583 

Generación # 2 
La mejor solución es: [0, 17, 36, 37, 14, 6, 5, 33, 30, 19, 11, 12, 2, 34, 21, 40, 27, 1, 29, 41, 15, 16, 32, 31, 35, 20, 28, 38, 22, 9, 10, 26, 3, 8, 13, 23, 18, 24, 39, 4, 25, 7] 
con distancia: 3583 

Generación # 3 
La mejor solución es: [0, 17, 36, 37, 14, 6, 5, 33, 30, 19, 11, 12, 2, 34, 21, 40, 27, 1, 29, 41, 15, 16, 32, 31, 35, 20, 28, 38, 22, 9, 10, 26, 3, 8, 13, 23, 18, 24, 39, 4, 25, 7] 
con distancia: 3583 

Generación # 4 
La mejor solución es: [0, 17, 36, 37, 14, 6, 27, 34, 33, 38, 23, 40, 24, 9, 11, 41, 13, 18, 16, 22, 39, 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

!jupyter nbconvert --to html "/content/drive/MyDrive/Colab Notebooks/Algoritmos_R3.ipynb"

from google.colab import files
files.download("/content/drive/MyDrive/Colab Notebooks/Algoritmos_R3.html")

Mounted at /content/drive
[NbConvertApp] Converting notebook /content/drive/MyDrive/Colab Notebooks/Algoritmos_R3.ipynb to html
[NbConvertApp] Writing 737498 bytes to /content/drive/MyDrive/Colab Notebooks/Algoritmos_R3.html


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>